# Final Assignment 

## Instuctions
* An extra 20 points for those who implement ALL the functions themselves without using tensorflows tf.keras.preprocessing and TextVectorization.
* Use pyarrow to save the embedding map.
* Find most similar function should be efficient. 20 sec to wait for a result is too much. You should aim to < 2 sec (use "timeit" magic to verify)
* Output cells should not be too big. DO NOT DUMP A LOT OF DATA IN THE OUTPUT. NOTEOOKS THAT WON'T FOLLOW THIS INSTRUCTION WILL NOT BE CHECKED AND THEIR GRADE WILL BE SET TO ZERO.

### Read dataset 

In [1]:
import zipfile
import pandas as pd

zip_path = 'data/review_230k.zip'

def extract_parquet_from_zip(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        file_list = zip_ref.namelist()
        print(f"Files in zip: {file_list}") 

        parquet_file_name = [f for f in file_list if f.endswith('.parquet')][0]

        with zip_ref.open(parquet_file_name) as file:
            df = pd.read_parquet(file)
    return df

data = extract_parquet_from_zip(zip_path)



Files in zip: ['review_230k.parquet']


In [2]:
def create_dataset(data: pd.DataFrame):
    """ Create the dataset in your preferrable format """
    return data

In [3]:
dataset = create_dataset(data)

#
print(dataset)

                                                title  \
0             Truly is "Jewel of the Upper Wets Side"   
1                             My home away from home!   
2                                          Great Stay   
3                                  Modern Convenience   
4       Its the best of the Andaz Brand in the US....   
...                                               ...   
230334                        Treated us like royalty   
230335        Fine time but some room for improvement   
230336                Great venue for corporate event   
230337                             Almost in the Loop   
230338                                    Comfortable   

                                                     text  
0       Stayed in a king suite for 11 nights and yes i...  
1       On every visit to NYC, the Hotel Beacon is the...  
2       This is a great property in Midtown. We two di...  
3       The Andaz is a nice hotel in a central locatio...  
4       I have 

In [4]:
first_title_value = dataset['text'].iloc[1]
print(dataset['title'].iloc[2])
print(dataset['text'].iloc[0])

Great Stay
Stayed in a king suite for 11 nights and yes it cots us a bit but we were happy with the standard of room, the location and the friendliness of the staff. Our room was on the 20th floor overlooking Broadway and the madhouse of the Fairway Market. Room was quite with no noise evident from the hallway or adjoining rooms. It was great to be able to open windows when we craved fresh rather than heated air. The beds, including the fold out sofa bed, were comfortable and the rooms were cleaned well. Wi-fi access worked like a dream with only one connectivity issue on our first night and this was promptly responded to with a call from the service provider to ensure that all was well. The location close to the 72nd Street subway station is great and the complimentary umbrellas on the drizzly days were greatly appreciated. It is fabulous to have the kitchen with cooking facilities and the access to a whole range of fresh foods directly across the road at Fairway.
This is the second t

### Clean and standardize the data

In [5]:

import string

def prepare_dataset(dataset):
    """ Clean and prepare the dataset before word encoding, returning a single 2D array """
    
    dataset['text'] = dataset['text'].fillna('')
    
    def clean_text(text):
        text = text.lower()
        sentences = text.split('.')
        cleaned_sentences = [
            sentence.translate(str.maketrans('', '', string.punctuation + string.digits)).strip()
            for sentence in sentences if sentence
        ]
        
        sentence_word_arrays = [sentence.split() for sentence in cleaned_sentences]
        return sentence_word_arrays
    dataset['tokenized_sentences'] = dataset['text'].apply(clean_text)
    all_tokenized_sentences = []
    for sentence_list in dataset['tokenized_sentences']:
        all_tokenized_sentences.extend(sentence_list)
    
    return all_tokenized_sentences

all_tokenized_sentences = prepare_dataset(dataset)

for sentence in all_tokenized_sentences[:10]:
    print(sentence) 


['stayed', 'in', 'a', 'king', 'suite', 'for', 'nights', 'and', 'yes', 'it', 'cots', 'us', 'a', 'bit', 'but', 'we', 'were', 'happy', 'with', 'the', 'standard', 'of', 'room', 'the', 'location', 'and', 'the', 'friendliness', 'of', 'the', 'staff']
['our', 'room', 'was', 'on', 'the', 'th', 'floor', 'overlooking', 'broadway', 'and', 'the', 'madhouse', 'of', 'the', 'fairway', 'market']
['room', 'was', 'quite', 'with', 'no', 'noise', 'evident', 'from', 'the', 'hallway', 'or', 'adjoining', 'rooms']
['it', 'was', 'great', 'to', 'be', 'able', 'to', 'open', 'windows', 'when', 'we', 'craved', 'fresh', 'rather', 'than', 'heated', 'air']
['the', 'beds', 'including', 'the', 'fold', 'out', 'sofa', 'bed', 'were', 'comfortable', 'and', 'the', 'rooms', 'were', 'cleaned', 'well']
['wifi', 'access', 'worked', 'like', 'a', 'dream', 'with', 'only', 'one', 'connectivity', 'issue', 'on', 'our', 'first', 'night', 'and', 'this', 'was', 'promptly', 'responded', 'to', 'with', 'a', 'call', 'from', 'the', 'service', 

### Create A vocabulary 

In [6]:
 
def create_vocabulary(all_tokenized_sentences) -> dict[str, int]:
 
    vocabulary = {'[UNK]': 0}
    index = 1
    for sentence in all_tokenized_sentences:
        for word in sentence:
            if word not in vocabulary:
                vocabulary[word] = index
                index += 1
    return vocabulary

all_tokenized_sentences = prepare_dataset(dataset)


vocabulary = create_vocabulary(all_tokenized_sentences)

n = 10
for i, (key, value) in enumerate(vocabulary.items()):
    if i >= n:
        break
    print(f"{key}: {value}")
print(type(vocabulary))

[UNK]: 0
stayed: 1
in: 2
a: 3
king: 4
suite: 5
for: 6
nights: 7
and: 8
yes: 9
<class 'dict'>


### Word encoding 

In [7]:
def encode_sentences(all_tokenized_sentences, vocabulary: dict[str, int]):
    """
    Encodes each word in all_tokenized_sentences based on the provided vocabulary.
    
    Parameters:
    - all_tokenized_sentences: List of tokenized sentences (each sentence is a list of words).
    - vocabulary: Dictionary mapping words to their corresponding integer indices.
    
    Returns:
    - encoded_sentences: List of encoded sentences (each sentence is a list of integer indices).
    """
    encoded_sentences = []
    for sentence in all_tokenized_sentences:
        encoded_sentence = [vocabulary.get(word, vocabulary.get('<UNK>')) for word in sentence]
        encoded_sentences.append(encoded_sentence)
    return encoded_sentences






In [8]:
encoded_sentences = encode_sentences(all_tokenized_sentences, vocabulary)
for encoded_sentence in encoded_sentences[:5]:
    print(encoded_sentence)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 3, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 19, 23, 8, 19, 24, 21, 19, 25]
[26, 22, 27, 28, 19, 29, 30, 31, 32, 8, 19, 33, 21, 19, 34, 35]
[22, 27, 36, 18, 37, 38, 39, 40, 19, 41, 42, 43, 44]
[10, 27, 45, 46, 47, 48, 46, 49, 50, 51, 15, 52, 53, 54, 55, 56, 57]
[19, 58, 59, 19, 60, 61, 62, 63, 16, 64, 8, 19, 44, 16, 65, 66]


Example using tensoflow 

In [9]:
from tensorflow.keras import layers
import tensorflow as tf

# Create a TensorFlow dataset from the 'text' column
text_ds = tf.data.Dataset.from_tensor_slices(dataset['text'])

# Create the vocabulary and convert it to a list
vocabulary = create_vocabulary(all_tokenized_sentences)
vocabulary_list = list(vocabulary.keys())

# Set up the TextVectorization layer
vectorize_layer = layers.TextVectorization(
    max_tokens=258000,
    output_mode='int',
    output_sequence_length=100,
    vocabulary=vocabulary_list
)

# Get the updated vocabulary
vocabulary = vectorize_layer.get_vocabulary()

AUTOTUNE = tf.data.experimental.AUTOTUNE

# Encode the text data
encoded_text_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()
encoded_dataset = encoded_text_ds.as_numpy_iterator()




In [10]:

print(f"Vocabulary size: {len(vocabulary)}")
# Convert the encoded dataset to a list
encoded_dataset = list(encoded_text_ds.as_numpy_iterator())
# Display a few encoded texts
for original_text, encoded_text in zip(dataset['text'], encoded_dataset[:5]):
    print(f"Original text: {original_text}")
    print(f"Encoded text: {encoded_text}")
    print("-" * 50)


Vocabulary size: 257009
Original text: Stayed in a king suite for 11 nights and yes it cots us a bit but we were happy with the standard of room, the location and the friendliness of the staff. Our room was on the 20th floor overlooking Broadway and the madhouse of the Fairway Market. Room was quite with no noise evident from the hallway or adjoining rooms. It was great to be able to open windows when we craved fresh rather than heated air. The beds, including the fold out sofa bed, were comfortable and the rooms were cleaned well. Wi-fi access worked like a dream with only one connectivity issue on our first night and this was promptly responded to with a call from the service provider to ensure that all was well. The location close to the 72nd Street subway station is great and the complimentary umbrellas on the drizzly days were greatly appreciated. It is fabulous to have the kitchen with cooking facilities and the access to a whole range of fresh foods directly across the road at F

### Generate Positive and Negative Pairs 

In [11]:
import random
from collections import defaultdict
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

def generate_training_data_optimized(encoded_dataset, number_of_neg_samples: int, window_size: int = 2):
    """
    Generate positive and negative pairs for Word2Vec training in an optimized way.

    Parameters:
    - encoded_dataset: List of lists, where each sublist contains word indices representing a sentence.
    - number_of_neg_samples: Number of negative samples per positive pair.
    - window_size: The context window size around the target word.

    Returns:
    - pairs: Numpy array with shape (num_samples, 2) containing target and context word indices.
    - labels: Numpy array with shape (num_samples,) containing labels (1 for positive pairs, 0 for negative pairs).
    """
    
    # Calculate word frequencies
    word_freq = defaultdict(int)
    for sentence in encoded_dataset:
        for word in sentence:
            word_freq[word] += 1

    total_words = sum(word_freq.values())

    # Calculate the probability distribution for negative sampling
    word_prob = np.array([(freq / total_words) ** 0.75 for freq in word_freq.values()])
    words = np.array(list(word_freq.keys()))
    word_prob /= word_prob.sum()

    # Generate positive pairs
    target_words, context_words = [], []
    for sentence in encoded_dataset:
        sentence_length = len(sentence)
        for i, target_word in enumerate(sentence):
            start = max(i - window_size, 0)
            end = min(i + window_size + 1, sentence_length)
            context = list(sentence[start:i]) + list(sentence[i + 1:end])
            target_words.extend([target_word] * len(context))
            context_words.extend(context)

    # Convert lists to NumPy arrays
    target_words = np.array(target_words)
    context_words = np.array(context_words)

    # Generate negative samples
    num_positive = len(target_words)
    negative_samples = np.random.choice(words, size=num_positive * number_of_neg_samples, p=word_prob)
    negative_samples = negative_samples.reshape(num_positive, number_of_neg_samples)

    # Create negative pairs
    target_neg = np.repeat(target_words, number_of_neg_samples)
    negative_words = negative_samples.flatten()

    # Combine positive and negative pairs
    pairs_positive = np.stack([target_words, context_words], axis=1)
    pairs_negative = np.stack([target_neg, negative_words], axis=1)

    pairs = np.concatenate([pairs_positive, pairs_negative], axis=0)
    labels = np.concatenate([np.ones(len(pairs_positive), dtype=np.int32), np.zeros(len(pairs_negative), dtype=np.int32)])

    return pairs, labels



# Example usage
number_of_neg_samples = 2
window_size = 2

pairs, labels = generate_training_data_optimized(encoded_dataset, number_of_neg_samples, window_size)

print(f"Total pairs generated: {len(pairs)}")
print(f"Example pair and label: {pairs[0]}, {labels[0]}")




Total pairs generated: 272260698
Example pair and label: [3 4], 1


In [12]:
import pyarrow as pa
import pyarrow.parquet as pq

# Function to save pairs and labels using PyArrow
def save_training_data_parquet(pairs: np.ndarray, labels: np.ndarray, file_path: str):
    """
    Saves pairs and labels to a file using PyArrow in Parquet format.
    
    Parameters:
    - pairs: Numpy array containing pairs (target, context/negative).
    - labels: Numpy array containing labels (1 for positive pairs, 0 for negative pairs).
    - file_path: The file path where data should be saved.
    """
    # Create a PyArrow table
    table = pa.Table.from_pydict({
        'target_word': pairs[:, 0],
        'context_word': pairs[:, 1],
        'label': labels
    })
    
    # Save the table in Parquet format
    pq.write_table(table, file_path)
    print(f"Training data saved to {file_path}")

training_data_file = 'training_data.parquet'
save_training_data_parquet(pairs, labels, training_data_file)


Training data saved to training_data.parquet


In [13]:
# Loading the saved data
loaded_table = pq.read_table('training_data.parquet')
loaded_df = loaded_table.to_pandas()

print(f"Loaded table shape: {loaded_df.shape}")
print(loaded_df.head())



Loaded table shape: (272260698, 3)
   target_word  context_word  label
0            3             4      1
1            3             5      1
2            4             3      1
3            4             5      1
4            4             6      1


In [14]:

# Create a reverse dictionary for decoding indices to words
index_to_word = {index: word for index, word in enumerate(vocabulary)}

def load_training_data_parquet(file_path: str):
    """
    Loads pairs and labels from a Parquet file.

    Parameters:
    - file_path: Path to the file containing the training data.

    Returns:
    - pairs: Numpy array of pairs (target, context/negative).
    - labels: Numpy array of labels (1 for positive pairs, 0 for negative pairs).
    """
    table = pq.read_table(file_path)
    df = table.to_pandas()
    pairs = df[['target_word', 'context_word']].values
    labels = df['label'].values
    return pairs, labels

def print_sample_pairs(pairs, labels, index_to_word, num_samples=5):
    """
    Displays a few word pairs with their decoding and labels.

    Parameters:
    - pairs: Numpy array of pairs (target, context/negative).
    - labels: Numpy array of labels (1 for positive pairs, 0 for negative pairs).
    - index_to_word: Reverse dictionary for decoding indices to words.
    - num_samples: Number of examples to display.
    """
    print("\nSample pairs with decoding:")
    for i in range(min(num_samples, len(pairs))):
        target_idx, context_idx = pairs[i]
        target_word = index_to_word.get(target_idx, '[UNK]')
        context_word = index_to_word.get(context_idx, '[UNK]')
        label = labels[i]
        pair_type = "Positive" if label == 1 else "Negative"
        print(f"{i+1}. {pair_type} pair: ({target_word}, {context_word})")

'''
training_data_file = 'training_data.parquet'
pairs, labels = load_training_data_parquet(training_data_file)
'''
print_sample_pairs(pairs, labels, index_to_word, num_samples=20)



Sample pairs with decoding:
1. Positive pair: (stayed, in)
2. Positive pair: (stayed, a)
3. Positive pair: (in, stayed)
4. Positive pair: (in, a)
5. Positive pair: (in, king)
6. Positive pair: (a, stayed)
7. Positive pair: (a, in)
8. Positive pair: (a, king)
9. Positive pair: (a, suite)
10. Positive pair: (king, in)
11. Positive pair: (king, a)
12. Positive pair: (king, suite)
13. Positive pair: (king, for)
14. Positive pair: (suite, a)
15. Positive pair: (suite, king)
16. Positive pair: (suite, for)
17. Positive pair: (suite, [UNK])
18. Positive pair: (for, king)
19. Positive pair: (for, suite)
20. Positive pair: (for, [UNK])


Example using tensoflow 

In [15]:
##### understand what is the functionalify of this, before you use it.




word_to_index = {word: index for index, word in enumerate(vocabulary)}
index_to_word = {idx: word for word, idx in word_to_index.items()}
VOCABULARY_SIZE = len(vocabulary)

sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(VOCABULARY_SIZE)
targets = []
contexts = []
labels = []

NUMBER_OF_NEGATIVE_SAMPLES=5

positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      encoded_dataset[0],
      vocabulary_size= VOCABULARY_SIZE,
      sampling_table=sampling_table,
      window_size=2,
      negative_samples=0,
      seed=42)


# Iterate over each positive skip-gram pair to produce training examples
# with a positive context word and negative samples.
for target_word, context_word in positive_skip_grams:
    
    context_class = tf.expand_dims(tf.constant([context_word], dtype="int64"), 1)
    negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=NUMBER_OF_NEGATIVE_SAMPLES,
          unique=True,
          range_max=VOCABULARY_SIZE,
          seed=123,
          name="negative_sampling")

    # Build context and label vectors (for one target word)
    context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
    label = tf.constant([1] + [0] * NUMBER_OF_NEGATIVE_SAMPLES, dtype="int64")

    # Append each element from the training example to global lists.
targets.append(target_word)
contexts.append(context)
labels.append(label)
targets = np.array(targets, dtype="int64")
contexts = np.array(contexts, dtype="int64")
labels = np.array(labels, dtype="int64")


dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(buffer_size=10000).batch(512, drop_remainder=True)


print(f"Number of training examples: {len(labels)}")
print(f"Sample pairs (target, context):\n{list(zip(targets, contexts))[:10]}")
print(f"Sample labels:\n{labels[:10]}")



Number of training examples: 1
Sample pairs (target, context):
[(62, array([  63,   68, 1423,    2,   10,   13], dtype=int64))]
Sample labels:
[[1 0 0 0 0 0]]


In [16]:
##### understand what is the functionalify of this, before you use it.




word_to_index = {word: index for index, word in enumerate(vocabulary)}
index_to_word = {idx: word for word, idx in word_to_index.items()}
VOCABULARY_SIZE = len(vocabulary)

sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(VOCABULARY_SIZE)

positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      encoded_dataset[0],
      vocabulary_size= VOCABULARY_SIZE,
      sampling_table=sampling_table,
      window_size=2,
      negative_samples=0,
      seed=42)

NUMBER_OF_NEGATIVE_SAMPLES=5
# Iterate over each positive skip-gram pair to produce training examples
# with a positive context word and negative samples.
for target_word, context_word in positive_skip_grams:
    
    context_class = tf.expand_dims(tf.constant([context_word], dtype="int64"), 1)
    negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
      true_classes=context_class,
      num_true=1,
      num_sampled=,
      unique=True,
      range_max=VOCABULARY_SIZE,
      seed=123,
      name="negative_sampling")

    # Build context and label vectors (for one target word)
    context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
    label = tf.constant([1] + [0] * NUMBER_OF_NEGATIVE_SAMPLES, dtype="int64")

    # Append each element from the training example to global lists.
    targets.append(target_word)
    contexts.append(context)
    labels.append(label)



SyntaxError: expected argument value expression (4224374398.py, line 29)

In [ ]:
##### understand what is the functionalify of this, before you use it.




word_to_index = {word: index for index, word in enumerate(vocabulary)}
index_to_word = {idx: word for word, idx in word_to_index.items()}
VOCABULARY_SIZE = len(vocabulary)

sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(VOCABULARY_SIZE)

positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      encoded_dataset[0],
      vocabulary_size= VOCABULARY_SIZE,
      sampling_table=sampling_table,
      window_size=2,
      negative_samples=0,
      seed=42)

NUMBER_OF_NEGATIVE_SAMPLES=5
# Iterate over each positive skip-gram pair to produce training examples
# with a positive context word and negative samples.
for target_word, context_word in positive_skip_grams:
    
    context_class = tf.expand_dims(tf.constant([context_word], dtype="int64"), 1)
    negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
      true_classes=context_class,
      num_true=1,
      num_sampled=,
      unique=True,
      range_max=VOCABULARY_SIZE,
      seed=123,
      name="negative_sampling")

    # Build context and label vectors (for one target word)
    context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
    label = tf.constant([1] + [0] * NUMBER_OF_NEGATIVE_SAMPLES, dtype="int64")

    # Append each element from the training example to global lists.
    targets.append(target_word)
    contexts.append(context)
    labels.append(label)



### Define the model 

In [ ]:
 # define you model here 

### Training

Example, there are other ways to train like using gradient tape

In [ ]:
model.compile(optimizer=..., loss=..., metrics=[...])

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="model_logs")

# checkpoint a model. here we save the best model relative to validation loss
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath="word2vec_model_w5_ns15_ckpt.h5", monitor=..., save_best_only=True)

# restore_best_weights - Whether to restore model weights from
# the epoch with the best value of the monitored quantity.
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, monitor='accuracy')

history = model.fit(dataset, epochs=..., callbacks=[tensorboard_callback, checkpoint_cb, early_stopping_cb])

model.save(f"word2vec_model_w{WINDOW_SIZE}_ns{NEGATIVE_SAMPLES}.h5", include_optimizer=True)

### Save embeddings 

In [ ]:

def save_embeddings(model, vocabulary: List[str], path: str):
    """ 
    Save the embeddings 
    param: model: the trained tf model 
    param: vocabulary: list of tokens 
    param: path: the path to save the embeddings map 

    hint: take the weights using model.get_layer(...).get_weights()
    """
    pass

### Find Most similar

Note that this must be efficient 

In [ ]:
def find_most_similar(word: str, k: int = 10) -> List[Tuple[str, float]]:
    """ 
    Find most similar tokens to word 
    param: word: the word to find most similar words to 
    k: number of most similar words 
    """
    pass

    

### Dimentionality Reduction 

visualize some clusters (pick some subset of words to show the labels for)